In [1]:
import pandas as pd
from my_functions import obrada, get_limits, generate_all_timestamps, make_table, check_holes_better,  filter_rows
import numpy as np
from matplotlib import pyplot as plt

In [18]:
df = pd.read_csv(r'postgres_public_measurements_dev.csv', parse_dates=['time'], encoding='latin1')
#odredivanje svih uredaja
devices = df['device'].unique()
columns = df.columns

print(columns)

Index(['id', 'time', 'energy', 'total_active_power',
       'demand_total_active_power', 'maximum_demand_total_active_power', 'i1',
       'i2', 'i3', 'u12', 'u23', 'u13', 'v1', 'v2', 'v3', 'device',
       'total_power_factor', 'total_apparent_power'],
      dtype='object')


In [3]:
#izrada csvova za svaki uredaj
for device in devices:
    df[df['device']==device].to_csv("{}_dev.csv".format(device), index=False)

In [4]:
df2 = pd.read_csv(r'postgres_public_measurements_prod.csv', parse_dates=['time'], encoding='latin1')
devices2 = df['device'].unique()
for device in devices2:
    df2[df2['device']==device].to_csv("{}_prod.csv".format(device), index=False)

In [29]:
#sad nekako spojit dev i prod po uredaju u jednu, novu tablicu
df = pd.read_csv(r'Aparat za kavu_dev.csv', parse_dates=['time'], encoding='latin1')

#najmanja udaljenost susjednih kolona, bilo što što je bliže trenutku od pola minimuma moze se 'poistovjetiti' kao isti trenutak

In [14]:
df['diff']=df['time'].diff(periods=-1)
minimum=df['diff'].min()
print(minimum)

0 days 00:00:01.985746


In [30]:
#sad nekako spojit dev i prod po uredaju u jednu, novu tablicu
df2 = pd.read_csv(r'Aparat za kavu_prod.csv', parse_dates=['time'], encoding='latin1')

#najmanja udaljenost susjednih kolona, bilo što što je bliže trenutku od pola minimuma moze se 'poistovjetiti' kao isti trenutak

In [ ]:
df2['diff']=df2['time'].diff(periods=-1)
minimum2=df2['diff'].min()
print(minimum2)

In [ ]:
df.drop(columns=['diff'], inplace=True)
df2.drop(columns=['diff'], inplace=True)

In [31]:
df=df.sort_values('time')
df2=df2.sort_values('time')

In [32]:
df=df.rename(columns={'time': 'time_dev'})
df2=df2.rename(columns={'time': 'time_prod'})

In [58]:
df

,id,time_dev,energy,total_active_power,demand_total_active_power,maximum_demand_total_active_power,i1,i2,i3,u12,u23,u13,v1,v2,v3,device,total_power_factor,total_apparent_power
299,1296352,2024-02-12 16:00:00.001248,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.000000,401.100006,400.500000,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
298,1296377,2024-02-12 16:00:02.001569,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.299988,401.500000,400.600006,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
297,1296402,2024-02-12 16:00:04.000326,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.299988,401.500000,400.600006,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
296,1296427,2024-02-12 16:00:06.001488,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.299988,401.299988,400.500000,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
295,1296452,2024-02-12 16:00:08.000475,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.299988,401.299988,400.500000,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1303727,2024-02-12 16:09:50.000329,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.600006,401.100006,400.200012,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
3,1303752,2024-02-12 16:09:52.000628,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.799988,401.299988,400.200012,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
2,1303777,2024-02-12 16:09:54.000515,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.799988,401.299988,400.200012,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
1,1303802,2024-02-12 16:09:56.000207,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.600006,401.200012,400.100006,NaN,NaN,NaN,Aparat za kavu,NaN,0.0


In [60]:
df2['time_prod'].duplicated().any()


False

In [54]:
merged = pd.merge_asof(df, df2, left_on="time_dev", right_on='time_prod', tolerance=pd.to_timedelta('00:00:01'), suffixes=('_dev', '_prod'), direction='nearest')
merged
#1s zato sto je frekvnecija zapisa lijeve tablice ispod 2s, to znači da ako je trenutak desnog zapisa udaljen od lijevog trenutka za više od 1s, pripada (odgovara) nekom drugom lijevom zapisu

,id_dev,time_dev,energy_dev,total_active_power_dev,demand_total_active_power_dev,maximum_demand_total_active_power_dev,i1_dev,i2_dev,i3_dev,u12_dev,...,i3_prod,u12_prod,u23_prod,u13_prod,v1_prod,v2_prod,v3_prod,device_prod,total_power_factor_prod,total_apparent_power_prod
0,1296352,2024-02-12 16:00:00.001248,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.000000,...,0.0,400.299988,401.500000,400.600006,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
1,1296377,2024-02-12 16:00:02.001569,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.299988,...,0.0,400.299988,401.500000,400.600006,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
2,1296402,2024-02-12 16:00:04.000326,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.299988,...,0.0,400.299988,401.299988,400.500000,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
3,1296427,2024-02-12 16:00:06.001488,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.299988,...,0.0,400.299988,401.299988,400.500000,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
4,1296452,2024-02-12 16:00:08.000475,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.299988,...,0.0,400.299988,401.299988,400.500000,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1303727,2024-02-12 16:09:50.000329,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.600006,...,0.0,400.799988,401.299988,400.200012,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
296,1303752,2024-02-12 16:09:52.000628,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.799988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,1303777,2024-02-12 16:09:54.000515,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.799988,...,0.0,400.799988,401.299988,400.200012,NaN,NaN,NaN,Aparat za kavu,NaN,0.0
298,1303802,2024-02-12 16:09:56.000207,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.600006,...,0.0,400.600006,401.200012,400.100006,NaN,NaN,NaN,Aparat za kavu,NaN,0.0


In [55]:
merged.sort_index(axis=1)

,demand_total_active_power_dev,demand_total_active_power_prod,device_dev,device_prod,energy_dev,energy_prod,i1_dev,i1_prod,i2_dev,i2_prod,...,u13_dev,u13_prod,u23_dev,u23_prod,v1_dev,v1_prod,v2_dev,v2_prod,v3_dev,v3_prod
0,287.283325,287.283325,Aparat za kavu,Aparat za kavu,10638286,10638.286,0.0,0.0,0.0,0.0,...,400.500000,400.600006,401.100006,401.500000,NaN,NaN,NaN,NaN,NaN,NaN
1,287.283325,287.283325,Aparat za kavu,Aparat za kavu,10638286,10638.286,0.0,0.0,0.0,0.0,...,400.600006,400.600006,401.500000,401.500000,NaN,NaN,NaN,NaN,NaN,NaN
2,287.283325,287.283325,Aparat za kavu,Aparat za kavu,10638286,10638.286,0.0,0.0,0.0,0.0,...,400.600006,400.500000,401.500000,401.299988,NaN,NaN,NaN,NaN,NaN,NaN
3,287.283325,287.283325,Aparat za kavu,Aparat za kavu,10638286,10638.286,0.0,0.0,0.0,0.0,...,400.500000,400.500000,401.299988,401.299988,NaN,NaN,NaN,NaN,NaN,NaN
4,287.283325,287.283325,Aparat za kavu,Aparat za kavu,10638286,10638.286,0.0,0.0,0.0,0.0,...,400.500000,400.500000,401.299988,401.299988,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,288.166687,288.166687,Aparat za kavu,Aparat za kavu,10638333,10638.333,0.0,0.0,0.0,0.0,...,400.200012,400.200012,401.100006,401.299988,NaN,NaN,NaN,NaN,NaN,NaN
296,288.166687,NaN,Aparat za kavu,NaN,10638333,NaN,0.0,NaN,0.0,NaN,...,400.200012,NaN,401.299988,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,288.166687,288.166687,Aparat za kavu,Aparat za kavu,10638333,10638.333,0.0,0.0,0.0,0.0,...,400.200012,400.200012,401.299988,401.299988,NaN,NaN,NaN,NaN,NaN,NaN
298,288.166687,288.166687,Aparat za kavu,Aparat za kavu,10638333,10638.333,0.0,0.0,0.0,0.0,...,400.100006,400.100006,401.200012,401.200012,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
merged[merged['time_prod'].duplicated()]
#ovih lijevih ima vise jer su precizniji, neki retci u desnom sl. jednostavno fale i zato su NaTovi

,id_dev,time_dev,energy_dev,total_active_power_dev,demand_total_active_power_dev,maximum_demand_total_active_power_dev,i1_dev,i2_dev,i3_dev,u12_dev,...,i3_prod,u12_prod,u23_prod,u13_prod,v1_prod,v2_prod,v3_prod,device_prod,total_power_factor_prod,total_apparent_power_prod
46,1297502,2024-02-12 16:01:32.000635,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,1298452,2024-02-12 16:02:48.001879,10638286,0.0,287.283325,38795.0625,0.0,0.0,0.0,400.899994,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,1299227,2024-02-12 16:03:50.000878,10638330,0.0,575.450012,38795.0625,0.0,0.0,0.0,400.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,1300127,2024-02-12 16:05:02.000267,10638333,0.0,575.450012,38795.0625,0.0,0.0,0.0,399.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,1301027,2024-02-12 16:06:14.003585,10638333,0.0,402.750000,38795.0625,0.0,0.0,0.0,399.200012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,1301952,2024-02-12 16:07:28.001582,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,399.600006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,1302877,2024-02-12 16:08:42.001010,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.700012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,1303752,2024-02-12 16:09:52.000628,10638333,0.0,288.166687,38795.0625,0.0,0.0,0.0,400.799988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#konacan kod
for device in devices:
    df = pd.read_csv(r'uredaji/{}_dev.csv'.format(device), parse_dates=['time'], encoding='latin1')
    df['diff'] = df['time'].diff(periods=-1)
    minimum = df['diff'].min()
    print(minimum)
    
    df2 = pd.read_csv(r'uredaji/{}_prod.csv'.format(device), parse_dates=['time'], encoding='latin1')
    df2['diff'] = df2['time'].diff(periods=-1)
    minimum2 = df2['diff'].min()
    print(minimum2)
    df2['energy']=df2['energy']*1000
    if minimum == pd.to_timedelta('0 days 00:00:00') or minimum2==pd.to_timedelta('0 days 00:00:00'):
        #postoje dupli retci, podijelit u 4 df-a, isti trenutak manji id u jednu tablicu, isti trenutka veci id u drugu
        df_gornji = df[df['diff']!=pd.to_timedelta('0 days 00:00:00')].copy()
        #u gronjoj su svi 'originali', u donjoj svi duplikati
        #isto za drugi dataframe
        df_donji = df[df['diff']==pd.to_timedelta('0 days 00:00:00')].copy()
        df2_gornji = df2[df2['diff']!=pd.to_timedelta('0 days 00:00:00')].copy()
        df2_donji = df2[df2['diff']==pd.to_timedelta('0 days 00:00:00')].copy()
        
        #spojit gornje međusobno, i donje međusobno
        df_gornji.drop(columns=['diff', 'id'], inplace=True)
        df_donji.drop(columns=['diff','id'], inplace=True)
        df2_donji.drop(columns=['diff','id'], inplace=True)
        df2_gornji.drop(columns=['diff','id'], inplace=True)
        
        df_gornji = df_gornji.sort_values(['time', 'energy'])
        df_donji = df_donji.sort_values(['time', 'energy'])
        df2_donji = df2_donji.sort_values(['time', 'energy'])
        df2_gornji = df2_gornji.sort_values(['time', 'energy'])
        
        df_gornji = df_gornji.rename(columns={'time': 'time_dev', 'energy': 'energy_dev'})
        df2_gornji = df2_gornji.rename(columns={'time': 'time_prod', 'energy': 'energy_prod'})
        df_donji = df_donji.rename(columns={'time': 'time_dev', 'energy': 'energy_dev'})
        df2_donji = df2_donji.rename(columns={'time': 'time_prod', 'energy': 'energy_prod'})
        
        merged_gornji = pd.merge_asof(df_gornji, df2_gornji, left_on="time_dev", right_on='time_prod', tolerance=pd.to_timedelta('00:00:01'), suffixes=('_dev', '_prod'), direction='nearest')
        merged_gornji = merged_gornji.sort_index(axis=1)
        
        merged_donji = pd.merge_asof(df_donji, df2_donji, left_on="time_dev", right_on='time_prod', tolerance=pd.to_timedelta('00:00:01'), suffixes=('_dev', '_prod'), direction='nearest')
        merged_donji = merged_donji.sort_index(axis=1)
        
        #u gornjem su svi 'originalni zapisi' koji međusbono odgovaraju, u donjem su sve kopije
        #sad treba sve skupa spojit u jednu tablicu
        merged = pd.concat([merged_gornji, merged_donji], ignore_index=True)
        merged = merged.sort_values(['time_dev'])
        
    else:   
        df.drop(columns=['diff','id'], inplace=True)
        df2.drop(columns=['diff','id'], inplace=True)
        
        df = df.sort_values(['time', 'energy'])
        df2 = df2.sort_values(['time', 'energy'])
        df = df.rename(columns={'time': 'time_dev', 'energy': 'energy_dev'})
        df2 = df2.rename(columns={'time': 'time_prod', 'energy': 'energy_prod'})
        
        print(df2.loc[:,['time_prod','energy_prod']].duplicated().any())
        
        merged = pd.merge_asof(df, df2, left_on="time_dev", right_on='time_prod', tolerance=pd.to_timedelta('00:00:01'), suffixes=('_dev', '_prod'), direction='nearest')
    merged = merged.sort_index(axis=1)
    merged.to_csv("merged\{}_merged.csv".format(device), index=False)

0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:00
0 days 00:00:00
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:00
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:01.985746
0 days 00:00:02
False
0 days 00:00:00
0 days 00:00:00


In [19]:
del df, df2, df_donji, merged_donji, merged_gornji, df2_donji, df2_gornji, df_gornji, merged

In [54]:
#za svaki merged file, za svako vrijeme pogledat razlike u istoimenim kolonama
#prvo trebam imena kolona, to mogu od gore, columns
for device in devices:
    #columns=columns.drop(['device'])
    df = pd.read_csv(r'merged/{}_merged.csv'.format(device), parse_dates=['time_dev', 'time_prod'], encoding='latin1')
    
    lst = []
    for i, row in df.iterrows():
        for column in columns:
            new_row = False
            value1 = row['{}_dev'.format(column)]
            value2 = row['{}_prod'.format(column)]
            if(not pd.isna(value1) or not pd.isna(value2)):
                #ako je bar jedna od vrijednosti razlicita od null
                if pd.isna(value1) or pd.isna(value2):
                    #ako je neka od njih nan automatski znaci da druga nije pa se moze odmah upisat
                    new_row=True
                elif column == 'demand_total_active_power' or column == 'v1' or column == 'maximum_demand_total_active_power' :
                    #ako value1 nije null i ako je zaokruzena razlicita od 6
                    if round(value1,6)!=value2:
                        new_row=True
                elif column == 'i1':
                    if value1!=round(value2,2):
                        new_row = True
                elif column == 'energy':
                    if value1!=round(value2):
                        new_row =True
                elif column == 'total_power_factor':
                    if round(value1,2)!=value2:
                        new_row = True
                else:
                    if row['{}_dev'.format(column)]!=row['{}_prod'.format(column)]:
                        new_row = True
                
            if new_row:
                #podatak nije isti, upisujem ga u novu tablicu
                diff = abs(value1-value2)
                lst.append({'time_dev':row['time_dev'], 'time_prod':row['time_prod'], 'value_dev':row['{}_dev'.format(column)], 'value_prod':row['{}_prod'.format(column)], 'diff':diff})
                
    final = pd.DataFrame(lst, columns=['time_dev', 'time_prod', 'value_dev', 'value_prod', 'diff'])     
    final.to_csv('final/{}_diff.csv'.format(device), index=False)

In [53]:
df = pd.read_csv(r'merged/{}_merged.csv'.format('Drobilica'), parse_dates=['time_dev', 'time_prod'], encoding='latin1')
df

,demand_total_active_power_dev,demand_total_active_power_prod,device_dev,device_prod,energy_dev,energy_prod,i1_dev,i1_prod,i2_dev,i2_prod,...,u13_dev,u13_prod,u23_dev,u23_prod,v1_dev,v1_prod,v2_dev,v2_prod,v3_dev,v3_prod
0,0.0,0.0,Drobilica,Drobilica,302,302.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,231.199997,231.199997,NaN,NaN,NaN,NaN
1,0.0,0.0,Drobilica,Drobilica,302,302.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,231.199997,231.199997,NaN,NaN,NaN,NaN
2,0.0,0.0,Drobilica,Drobilica,302,302.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,231.199997,231.100006,NaN,NaN,NaN,NaN
3,0.0,0.0,Drobilica,Drobilica,302,302.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,231.100006,231.100006,NaN,NaN,NaN,NaN
4,0.0,0.0,Drobilica,Drobilica,302,302.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,231.100006,231.100006,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.0,0.0,Drobilica,Drobilica,302,302.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,230.399994,230.500000,NaN,NaN,NaN,NaN
296,0.0,NaN,Drobilica,NaN,302,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,230.500000,NaN,NaN,NaN,NaN,NaN
297,0.0,0.0,Drobilica,Drobilica,302,302.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,230.500000,230.500000,NaN,NaN,NaN,NaN
298,0.0,0.0,Drobilica,Drobilica,302,302.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,230.500000,230.500000,NaN,NaN,NaN,NaN
